In [ ]:
#Se importan librerías iniciales
!pip install rlxutils
import os
import pandas as pd
import numpy as np
from rlxutils import subplots
import matplotlib.pyplot as plt

#Se importan otras librerías requeridas
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


#Se cargan los datos de Kagle
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

#Se extraen y descomprimen archivos
!unzip udea*.zip > /dev/null

#Se carga el archivo "train.csv"
GetFileTrain = pd.read_csv("train.csv")

#IMPORTANTE!!
#Se excluye la columna "ID" en el análisis
#DataSet = Data
Data = GetFileTrain.drop(columns=['ID'])

#COLUMNA PERIODO
print("COLUMNA PERIODO")
#Modificar la columna 'PERIODO' para que solo contenga el año
Data['PERIODO'] = Data['PERIODO'].astype(str).str[:4].astype(int)
#Diccionario de mapeo para los vaAalores de desempeño según PERIODO
desempeno_periodo_map = {
    2018: 1,
    2019: 0,
    2020: 1,
    2021: 0
}
#Crear la columna DESEMPEÑO_PERIODO basada en los valores de PERIODO
Data['DESEMPEÑO_PERIODO'] = Data['PERIODO'].map(desempeno_periodo_map)
Data['PEOR_DESEMPEÑO_PERIODO'] = 1 - Data['DESEMPEÑO_PERIODO']
#Verificar las primeras filas
print(Data[['PERIODO', 'DESEMPEÑO_PERIODO']].head(10))
#Crear columnas binarias para cada año en la columna 'PERIODO' con prefijo 'PERIODO'
Data = pd.get_dummies(Data, columns=['PERIODO'], prefix='PERIODO')
#Filtrar las columnas que empiezan con 'PERIODO_'
periodo_columns = [col for col in Data.columns if col.startswith('PERIODO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[periodo_columns] = Data[periodo_columns].astype(int)
#Verificar los cambios
for col in periodo_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")


#COLUMNA ESTU_PRGM_ACADEMICO
print("COLUMNA ESTU_PRGM_ACADEMICO")
import unicodedata
#Función para eliminar tildes y normalizar el texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.upper()
        texto = ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))
    return texto
#Cargar el archivo Excel "Programas.xlsx" y seleccionar la hoja "Sheet1"
programas_df = pd.read_excel('Programas.xlsx', sheet_name='Sheet1')
#Asegurarse de que los datos en la columna "ESTU_PRGM_ACADEMICO" estén normalizados
Data['ESTU_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].apply(normalizar_texto)
#Normalizar también la columna 'Programa' y 'Campo de Conocimiento' en el archivo Excel
programas_df['Programa'] = programas_df['Programa'].apply(normalizar_texto)
programas_df['Campo de Conocimiento'] = programas_df['Campo de Conocimiento'].apply(normalizar_texto)
#Crear un diccionario para coincidencias exactas entre 'Programa' y 'Campo de Conocimiento'
reemplazos = dict(zip(programas_df['Programa'], programas_df['Campo de Conocimiento']))
#Reemplazar las coincidencias exactas en la columna 'ESTU_PRGM_ACADEMICO'
Data['ESTU_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].replace(reemplazos)
#Verificar los primeros valores después del reemplazo
print(Data['ESTU_PRGM_ACADEMICO'].head(10))
# Diccionario de mapeo para 'DESEMPEÑO_PRGM_ACADEMICO'
desempeno_prgm_map = {
    'SALUD': 1,
    'CIENCIAS SOCIALES Y HUMANIDADES': 0,
    'CIENCIAS ECONOMICAS Y ADMINISTRATIVAS': 0,
    'AGRONOMIA, VETERINARIA Y AFINES': 1,
    'CIENCIAS NATURALES E INGENIERIA': 1,
    'EDUCACION Y PEDAGOGIA': 0,
    'ARTE, DISENO, CULTURA Y DEPORTE': 1
}
#Crear la nueva columna 'DESEMPEÑO_PRGM_ACADEMICO' basada en 'ESTU_PRGM_ACADEMICO'
Data['DESEMPEÑO_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].map(desempeno_prgm_map)
Data['PEOR_DESEMPEÑO_PRGM_ACADEMICO'] = 1 - Data['DESEMPEÑO_PRGM_ACADEMICO']
#Verificar las primeras filas
print(Data[['ESTU_PRGM_ACADEMICO', 'DESEMPEÑO_PRGM_ACADEMICO']].head(10))
#Crear columnas binarias para cada programa en 'ESTU_PRGM_ACADEMICO' con prefijo 'PRGM_'
Data = pd.get_dummies(Data, columns=['ESTU_PRGM_ACADEMICO'], prefix='PRGM')
#Filtrar las columnas que empiezan con 'PRGM_'
program_columns = [col for col in Data.columns if col.startswith('PRGM_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[program_columns] = Data[program_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in program_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD
print("COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD")
Data['ESTU_VALORMATRICULAUNIVERSIDAD'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_VLRMATRICULA'
desempeno_vlrmatricula_map = {
    'No pagó matrícula': 0,
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 0,
    'Entre 1 millón y menos de 2.5 millones': 0,
    'Entre 2.5 millones y menos de 4 millones': 0,
    'Entre 4 millones y menos de 5.5 millones': 1,
    'Entre 5.5 millones y menos de 7 millones': 1,
    'Más de 7 millones': 1,
    'ValorNulo': 1
}
#Reemplazar valores nulos con 1
Data['ESTU_VALORMATRICULAUNIVERSIDAD'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(1)
#Crear la nueva columna 'DESEMPEÑO_VLRMATRICULA' basada en 'ESTU_VALORMATRICULAUNIVERSIDAD'
Data['DESEMPEÑO_VLRMATRICULA'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(desempeno_vlrmatricula_map)
Data['PEOR_DESEMPEÑO_VLRMATRICULA'] = 1 - Data['DESEMPEÑO_VLRMATRICULA']
#Verificar las primeras filas
print(Data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'DESEMPEÑO_VLRMATRICULA']].head(10))
#Reemplazar los valores en 'ESTU_VALORMATRICULAUNIVERSIDAD' para simplificar
Data['ESTU_VALORMATRICULAUNIVERSIDAD'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
    'No pagó matrícula': "Nivel0",
    'Menos de 500 mil': "Nivel1",
    'Entre 500 mil y menos de 1 millón': "Nivel2",
    'Entre 1 millón y menos de 2.5 millones': "Nivel3",
    'Entre 2.5 millones y menos de 4 millones': "Nivel4",
    'Entre 4 millones y menos de 5.5 millones': "Nivel5",
    'Entre 5.5 millones y menos de 7 millones': "Nivel6",
    'Más de 7 millones': "Nivel7",
    'ValorNulo': 'ValorNulo'
})
#Crear columnas binarias para cada nivel de 'ESTU_VALORMATRICULAUNIVERSIDAD' con prefijo 'VLRMATRICULA_'
Data = pd.get_dummies(Data, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='VLRMATRICULA')
#Filtrar las columnas que empiezan con 'VLRMATRICULA_'
vlrmatricula_columns = [col for col in Data.columns if col.startswith('VLRMATRICULA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[vlrmatricula_columns] = Data[vlrmatricula_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in vlrmatricula_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA DESEMPEÑO_DPTO
print("COLUMNA DESEMPEÑO_DPTO")
#Diccionario de mapeo para 'DESEMPEÑO_DPTO'
desempeno_dpto_map = {
    'ATLANTICO': 1,
    'BOLIVAR': 1,
    'CESAR': 0,
    'CORDOBA': 0,
    'LA GUAJIRA': 0,
    'MAGDALENA': 0,
    'SUCRE': 0,
    'SAN ANDRES': 0,
    'ANTIOQUIA': 1,
    'BOYACA': 1,
    'CALDAS': 1,
    'CUNDINAMARCA': 1,
    'HUILA': 0,
    'NORTE SANTANDER': 0,
    'QUINDIO': 1,
    'RISARALDA': 0,
    'SANTANDER': 1,
    'TOLIMA': 0,
    'BOGOTÁ': 1,
    'CAUCA': 0,
    'CHOCO': 0,
    'NARIÑO': 1,
    'VALLE': 1,
    'ARAUCA': 0,
    'CASANARE': 0,
    'META': 1,
    'VICHADA': 0,
    'AMAZONAS': 0,
    'CAQUETA': 0,
    'GUAINIA': 0,
    'GUAVIARE': 0,
    'PUTUMAYO': 0,
    'VAUPES': 0
}
#Crear la nueva columna 'DESEMPEÑO_DPTO' basada en 'ESTU_PRGM_DEPARTAMENTO'
Data['DESEMPEÑO_DPTO'] = Data['ESTU_PRGM_DEPARTAMENTO'].map(desempeno_dpto_map)
Data['PEOR_DESEMPEÑO_DPTO'] = 1 - Data['DESEMPEÑO_DPTO']
#Verificar las primeras filas
print(Data[['ESTU_PRGM_DEPARTAMENTO', 'DESEMPEÑO_DPTO']].head(10))
#Crear columnas binarias para cada departamento en 'ESTU_PRGM_DEPARTAMENTO' con prefijo 'DPTO_'
Data = pd.get_dummies(Data, columns=['ESTU_PRGM_DEPARTAMENTO'], prefix='DPTO')
#iltrar las columnas que empiezan con 'DPTO_'
dpto_columns = [col for col in Data.columns if col.startswith('DPTO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[dpto_columns] = Data[dpto_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in dpto_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_HORASSEMANATRABAJA
print("COLUMNA ESTU_HORASSEMANATRABAJA")
Data['ESTU_HORASSEMANATRABAJA'] = Data['ESTU_HORASSEMANATRABAJA'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_HTRABAJO'
desempeno_htrabajo_map = {
    '0': 1,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 1,
    'Entre 21 y 30 horas': 1,
    'Más de 30 horas': 0,
    'ValorNulo': 0
}
#Crear la nueva columna 'DESEMPEÑO_HTRABAJO' basada en 'ESTU_HORASSEMANATRABAJA'
Data['DESEMPEÑO_HTRABAJO'] = Data['ESTU_HORASSEMANATRABAJA'].map(desempeno_htrabajo_map)
Data['PEOR_DESEMPEÑO_HTRABAJO'] = 1 - Data['DESEMPEÑO_HTRABAJO']
#Verificar las primeras filas
print(Data[['ESTU_HORASSEMANATRABAJA', 'DESEMPEÑO_HTRABAJO']].head(10))
#Crear columnas binarias para cada rango de horas en 'ESTU_HORASSEMANATRABAJA' con prefijo 'HTRABAJO_'
Data = pd.get_dummies(Data, columns=['ESTU_HORASSEMANATRABAJA'], prefix='HTRABAJO')
#Filtrar las columnas que empiezan con 'HTRABAJO_'
htrabajo_columns = [col for col in Data.columns if col.startswith('HTRABAJO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[htrabajo_columns] = Data[htrabajo_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in htrabajo_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")

print("--------------------------------------------------------")

#COLUMNA FAMI_ESTRATOVIVIENDA
print("COLUMNA FAMI_ESTRATOVIVIENDA")
Data['FAMI_ESTRATOVIVIENDA'] = Data['FAMI_ESTRATOVIVIENDA'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_ESTRATO'
desempeno_estrato_map = {
    'Sin Estrato': 0,
    'Estrato 1': 0,
    'Estrato 2': 0,
    'Estrato 3': 1,
    'Estrato 4': 1,
    'Estrato 5': 1,
    'Estrato 6': 1,
    'ValorNulo': 0
}
#Crear la nueva columna 'DESEMPEÑO_ESTRATO' basada en 'FAMI_ESTRATOVIVIENDA'
Data['DESEMPEÑO_ESTRATO'] = Data['FAMI_ESTRATOVIVIENDA'].map(desempeno_estrato_map)
Data['PEOR_DESEMPEÑO_ESTRATO'] = 1 - Data['DESEMPEÑO_ESTRATO']
#Verificar las primeras filas
print(Data[['FAMI_ESTRATOVIVIENDA', 'DESEMPEÑO_ESTRATO']].head(10))
#Crear columnas binarias para cada nivel en 'FAMI_ESTRATOVIVIENDA' sin prefijo adicional
Data = pd.get_dummies(Data, columns=['FAMI_ESTRATOVIVIENDA'])
#Filtrar las columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_'
estrato_columns = [col for col in Data.columns if col.startswith('FAMI_ESTRATOVIVIENDA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[estrato_columns] = Data[estrato_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in estrato_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_TIENEINTERNET
print("COLUMNA FAMI_TIENEINTERNET")
Data['FAMI_TIENEINTERNET'] = Data['FAMI_TIENEINTERNET'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_INTERNET'
desempeno_internet_map = {
    'Si': 1,
    'No': 0,
    'ValorNulo': 0
}
#Crear la nueva columna 'DESEMPEÑO_INTERNET' basada en 'FAMI_TIENEINTERNET'
Data['DESEMPEÑO_INTERNET'] = Data['FAMI_TIENEINTERNET'].map(desempeno_internet_map)
Data['PEOR_DESEMPEÑO_INTERNET'] = 1 - Data['DESEMPEÑO_INTERNET']
#Verificar las primeras filas
print(Data[['FAMI_TIENEINTERNET', 'DESEMPEÑO_INTERNET']].head(10))
 #Crear columnas binarias para cada valor en 'FAMI_TIENEINTERNET' con prefijo 'INTERNET'
Data = pd.get_dummies(Data, columns=['FAMI_TIENEINTERNET'], prefix='INTERNET')
#Filtrar las columnas que empiezan con 'INTERNET_'
internet_columns = [col for col in Data.columns if col.startswith('INTERNET_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[internet_columns] = Data[internet_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in internet_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNAS AMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
#Diccionario de reemplazos para las columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
reemplazos_educacion = {
    'Técnica o tecnológica incompleta': 'TTIncompleta',
    'Técnica o tecnológica completa': 'TTCompleta',
    'Secundaria (Bachillerato) completa': 'SecCompleta',
    'No sabe': 'NA',
    'Primaria completa': 'Pcompleta',
    'Educación profesional completa': 'ProfCompleta',
    'Educación profesional incompleta': 'ProfIncompleta',
    'Primaria incompleta': 'PIncompleta',
    'Postgrado': 'Postgrado',
    'Secundaria (Bachillerato) incompleta': 'SecIncompleta',
    'Ninguno': 'NA',
    'No Aplica': 'NA',
    np.nan: 'ValorNulo'  # Reemplazo para valores nulos
}

#Aplicar los reemplazos en ambas columnas
Data['FAMI_EDUCACIONPADRE'] = Data['FAMI_EDUCACIONPADRE'].replace(reemplazos_educacion)
Data['FAMI_EDUCACIONMADRE'] = Data['FAMI_EDUCACIONMADRE'].replace(reemplazos_educacion)
#Crear columnas binarias para 'FAMI_EDUCACIONPADRE' con prefijo 'EDUPADRE_'
Data = pd.get_dummies(Data, columns=['FAMI_EDUCACIONPADRE'], prefix='EDUPADRE')
#Crear columnas binarias para 'FAMI_EDUCACIONMADRE' con prefijo 'EDUMADRE_'
Data = pd.get_dummies(Data, columns=['FAMI_EDUCACIONMADRE'], prefix='EDUMADRE')
#Filtrar las columnas que empiezan con 'EDUPADRE_' y 'EDUMADRE_'
edupadre_columns = [col for col in Data.columns if col.startswith('EDUPADRE_')]
edumadre_columns = [col for col in Data.columns if col.startswith('EDUMADRE_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[edupadre_columns] = Data[edupadre_columns].astype(int)
Data[edumadre_columns] = Data[edumadre_columns].astype(int)
# Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONPADRE'
print("Columnas binarias para FAMI_EDUCACIONPADRE:")
for col in edupadre_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
# Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONMADRE'
print("Columnas binarias para FAMI_EDUCACIONMADRE:")
for col in edumadre_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PAGOMATRICULAPROPIO
print("COLUMNA ESTU_PAGOMATRICULAPROPIO")
Data['ESTU_PAGOMATRICULAPROPIO'] = Data['ESTU_PAGOMATRICULAPROPIO'].fillna('ValorNulo')
# Diccionario de mapeo para 'DESEMPEÑO_PAGOM'
desempeno_pagom_map = {
    'Si': 0,
    'No': 1,
    'ValorNulo': 1
}
# Crear la nueva columna 'DESEMPEÑO_PAGOM' basada en 'ESTU_PAGOMATRICULAPROPIO'
Data['DESEMPEÑO_PAGOM'] = Data['ESTU_PAGOMATRICULAPROPIO'].map(desempeno_pagom_map)
Data['PEOR_DESEMPEÑO_PAGOM'] = 1 - Data['DESEMPEÑO_PAGOM']
# Verificar las primeras filas
print(Data[['ESTU_PAGOMATRICULAPROPIO', 'DESEMPEÑO_PAGOM']].head(10))
# Crear columnas binarias para cada valor en 'ESTU_PAGOMATRICULAPROPIO' con prefijo 'MLAPROPIO'
Data = pd.get_dummies(Data, columns=['ESTU_PAGOMATRICULAPROPIO'], prefix='MLAPROPIO')
# Filtrar las columnas que empiezan con 'MLAPROPIO_'
mlapropio_columns = [col for col in Data.columns if col.startswith('MLAPROPIO_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[mlapropio_columns] = Data[mlapropio_columns].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in mlapropio_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA RENDIMIENTO_GLOBAL
print("COLUMNA RENDIMIENTO_GLOBAL")
#Crear un diccionario de reemplazo para los valores de RENDIMIENTO_GLOBAL
reemplazos_rendimiento = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4
}
#Reemplazar los valores en la columna RENDIMIENTO_GLOBAL
Data['RENDIMIENTO_GLOBAL'] = Data['RENDIMIENTO_GLOBAL'].replace(reemplazos_rendimiento)
#Verificar los primeros valores después del reemplazo
print(Data['RENDIMIENTO_GLOBAL'].head())
print("--------------------------------------------------------")




  Preparing metadata (setup.py) ... done
  Created wheel for rlxutils: filename=rlxutils-0.1.10-py3-none-any.whl size=11100 sha256=a4cf96a645d9a27151c09271bcb6f1e881257f984d854b9b9456981ef69fae27
  Stored in directory: /root/.cache/pip/wheels/9a/45/da/49bdb0e82cc7a605e9c05dc24265687c5f349b53dcb74728e2
Successfully built rlxutils
 65% 13.0M/20.1M [00:00<00:00, 62.3MB/s]
100% 20.1M/20.1M [00:00<00:00, 81.0MB/s]
COLUMNA PERIODO
   PERIODO  DESEMPEÑO_PERIODO
0     2021                  0
1     2021                  0
2     2020                  1
3     2019                  0
4     2021                  0
5     2020                  1
6     2018                  1
7     2018                  1
8     2021                  0
9     2018                  1
PERIODO_2018:
 0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    1
8    0
9    1
Name: PERIODO_2018, dtype: int64

----------------------------------------

PERIODO_2019:
 0    0
1    0
2    0
3    1
4    0
5    0
6    0
7    0
8    0
9 

<ipython-input-1-dcbc921c785b>:380: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data['RENDIMIENTO_GLOBAL'] = Data['RENDIMIENTO_GLOBAL'].replace(reemplazos_rendimiento)


In [ ]:
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier

# Preparar los datos
# Excluir columnas DESEMPEÑO_
binary_columns = [col for col in Data.columns if col not in ['RENDIMIENTO_GLOBAL']]

#Filtrar las columnas binarias para el modelo
X = Data[binary_columns]
y = Data['RENDIMIENTO_GLOBAL']  # Variable objetivo

#Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el modelo con CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    depth=8,
    learning_rate=0.05,
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train)

# Evaluar el modelo
# Hacer predicciones
y_pred = model.predict(X_test)

#Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Precisión del modelo (CatBoost):", accuracy)
print("Reporte de clasificación:\n", report)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00
0:	learn: 1.3760296	total: 535ms	remaining: 8m 54s
100:	learn: 1.2407032	total: 54.7s	remaining: 8m 6s
200:	learn: 1.2274504	total: 1m 51s	remaining: 7m 24s
300:	learn: 1.2197643	total: 2m 44s	remaining: 6m 21s
400:	learn: 1.2139223	total: 3m 40s	remaining: 5m 28s
500:	learn: 1.2091065	total: 4m 31s	remaining: 4m 30s
600:	learn: 1.2048765	total: 5m 25s	remaining: 3m 36s
700:	learn: 1.2011083	total: 6m 21s	remaining: 2m 42s
800:	learn: 1.1974907	total: 7m 13s	remaining: 1m 47s
900:	learn: 1.1940581	total: 8m 8s	remaining: 53.7s
999:	learn: 1.1908041	total: 9m 2s	remaining: 0us
Precisión del modelo (CatBoost): 0.4237930204572804
Reporte de clasificación:
               precision    recall  f1-score   support

           1       0.44      0.56      0.49     51767
           2       0.33      0.26      0.29     51712
           3       0.32      0.25      0.28     51439
           4       0.53      0.62      0.57

In [ ]:
# Cargar el archivo test.csv
zt = pd.read_csv("test.csv")
# Conservar los IDs para el archivo de envío
zt_ids = zt['ID'].values

#COLUMNA PERIODO
print("COLUMNA PERIODO")
# Modificar la columna 'PERIODO' en test.csv para que solo contenga el año
zt['PERIODO'] = zt['PERIODO'].astype(str).str[:4].astype(int)
zt['DESEMPEÑO_PERIODO'] = zt['PERIODO'].map(desempeno_periodo_map)
zt['PEOR_DESEMPEÑO_PERIODO'] = 1 - Data['DESEMPEÑO_PERIODO']
# Crear columnas binarias para cada año en la columna 'PERIODO' con prefijo 'PERIODO'
zt = pd.get_dummies(zt, columns=['PERIODO'], prefix='PERIODO')
# Filtrar las columnas que empiezan con 'PERIODO_'
periodo_columns_test = [col for col in zt.columns if col.startswith('PERIODO_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[periodo_columns_test] = zt[periodo_columns_test].astype(int)
# Verificar los cambios
for col in periodo_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PRGM_ACADEMICO
print("COLUMNA ESTU_PRGM_ACADEMICO")
import unicodedata
import pandas as pd
# Función para eliminar tildes y normalizar el texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.upper()
        texto = ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))
    return texto
# Cargar el archivo Excel "Programas.xlsx" y seleccionar la hoja "Sheet1"
programas_df = pd.read_excel('Programas.xlsx', sheet_name='Sheet1')
# Asegurarse de que los datos en la columna "ESTU_PRGM_ACADEMICO" estén normalizados
zt['ESTU_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].apply(normalizar_texto)
# Normalizar también la columna 'Programa' y 'Campo de Conocimiento' en el archivo Excel
programas_df['Programa'] = programas_df['Programa'].apply(normalizar_texto)
programas_df['Campo de Conocimiento'] = programas_df['Campo de Conocimiento'].apply(normalizar_texto)
# Crear un diccionario para coincidencias exactas entre 'Programa' y 'Campo de Conocimiento'
reemplazos = dict(zip(programas_df['Programa'], programas_df['Campo de Conocimiento']))
# Reemplazar las coincidencias exactas en la columna 'ESTU_PRGM_ACADEMICO'
zt['ESTU_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].replace(reemplazos)
# Verificar los primeros valores después del reemplazo
print(zt['ESTU_PRGM_ACADEMICO'].head(10))
# Imprimir todos los valores únicos de la columna ESTU_PRGM_ACADEMICO
valores_unicos_test = zt['ESTU_PRGM_ACADEMICO'].unique()
print("Valores únicos en ESTU_PRGM_ACADEMICO (test.csv):")
print(valores_unicos_test)
zt['DESEMPEÑO_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].map(desempeno_prgm_map)
zt['PEOR_DESEMPEÑO_PRGM_ACADEMICO'] = 1 - Data['DESEMPEÑO_PRGM_ACADEMICO']
# Crear columnas binarias para cada programa en 'ESTU_PRGM_ACADEMICO' con prefijo 'PRGM_'
zt = pd.get_dummies(zt, columns=['ESTU_PRGM_ACADEMICO'], prefix='PRGM')
# Filtrar las columnas que empiezan con 'PRGM_'
program_columns_test = [col for col in zt.columns if col.startswith('PRGM_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[program_columns_test] = zt[program_columns_test].astype(int)
# verificar los cambios en las primeras 10 filas de cada nueva columna
for col in program_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD
print("COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD")
zt['ESTU_VALORMATRICULAUNIVERSIDAD'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('ValorNulo')
zt['DESEMPEÑO_VLRMATRICULA'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].map(desempeno_vlrmatricula_map)
zt['PEOR_DESEMPEÑO_VLRMATRICULA'] = 1 - Data['DESEMPEÑO_VLRMATRICULA']
# Reemplazar los valores en 'ESTU_VALORMATRICULAUNIVERSIDAD' para simplificar en test.csv
zt['ESTU_VALORMATRICULAUNIVERSIDAD'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
    'No pagó matrícula': "Nivel0",
    'Menos de 500 mil': "Nivel1",
    'Entre 500 mil y menos de 1 millón': "Nivel2",
    'Entre 1 millón y menos de 2.5 millones': "Nivel3",
    'Entre 2.5 millones y menos de 4 millones': "Nivel4",
    'Entre 4 millones y menos de 5.5 millones': "Nivel5",
    'Entre 5.5 millones y menos de 7 millones': "Nivel6",
    'Más de 7 millones': "Nivel7",
    'ValorNulo': 'ValorNulo'
})
# Crear columnas binarias para cada nivel de 'ESTU_VALORMATRICULAUNIVERSIDAD' con prefijo 'VLRMATRICULA_'
zt = pd.get_dummies(zt, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='VLRMATRICULA')
# Filtrar las columnas que empiezan con 'VLRMATRICULA_'
vlrmatricula_columns_test = [col for col in zt.columns if col.startswith('VLRMATRICULA_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[vlrmatricula_columns_test] = zt[vlrmatricula_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in vlrmatricula_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA DESEMPEÑO_DPTO
print("COLUMNA DESEMPEÑO_DPTO")
zt['DESEMPEÑO_DPTO'] = zt['ESTU_PRGM_DEPARTAMENTO'].map(desempeno_dpto_map)
zt['PEOR_DESEMPEÑO_DPTO'] = 1 - Data['DESEMPEÑO_DPTO']
# Crear columnas binarias para cada departamento en 'ESTU_PRGM_DEPARTAMENTO' con prefijo 'DPTO_'
zt = pd.get_dummies(zt, columns=['ESTU_PRGM_DEPARTAMENTO'], prefix='DPTO')
# Filtrar las columnas que empiezan con 'DPTO_'
dpto_columns_test = [col for col in zt.columns if col.startswith('DPTO_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[dpto_columns_test] = zt[dpto_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in dpto_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_HORASSEMANATRABAJA
print("COLUMNA ESTU_HORASSEMANATRABAJA")
zt['ESTU_HORASSEMANATRABAJA'] = zt['ESTU_HORASSEMANATRABAJA'].fillna('ValorNulo')
zt['DESEMPEÑO_HTRABAJO'] = zt['ESTU_HORASSEMANATRABAJA'].map(desempeno_htrabajo_map)
zt['PEOR_DESEMPEÑO_HTRABAJO'] = 1 - Data['DESEMPEÑO_HTRABAJO']
# Crear columnas binarias para cada rango de horas en 'ESTU_HORASSEMANATRABAJA' con prefijo 'HTRABAJO_'
zt = pd.get_dummies(zt, columns=['ESTU_HORASSEMANATRABAJA'], prefix='HTRABAJO')
# Filtrar las columnas que empiezan con 'HTRABAJO_'
htrabajo_columns_test = [col for col in zt.columns if col.startswith('HTRABAJO_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[htrabajo_columns_test] = zt[htrabajo_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in htrabajo_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_ESTRATOVIVIENDA
print("COLUMNA FAMI_ESTRATOVIVIENDA")
zt['FAMI_ESTRATOVIVIENDA'] = zt['FAMI_ESTRATOVIVIENDA'].fillna('ValorNulo')
zt['DESEMPEÑO_ESTRATO'] = zt['FAMI_ESTRATOVIVIENDA'].map(desempeno_estrato_map)
zt['PEOR_DESEMPEÑO_ESTRATO'] = 1 - Data['DESEMPEÑO_ESTRATO']

# Crear columnas binarias para cada nivel en 'FAMI_ESTRATOVIVIENDA' sin prefijo adicional
zt = pd.get_dummies(zt, columns=['FAMI_ESTRATOVIVIENDA'])
# Filtrar las columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_'
estrato_columns_test = [col for col in zt.columns if col.startswith('FAMI_ESTRATOVIVIENDA_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[estrato_columns_test] = zt[estrato_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in estrato_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_TIENEINTERNET
print("COLUMNA FAMI_TIENEINTERNET")
zt['FAMI_TIENEINTERNET'] = zt['FAMI_TIENEINTERNET'].fillna('ValorNulo')
zt['DESEMPEÑO_INTERNET'] = zt['FAMI_TIENEINTERNET'].map(desempeno_internet_map)
zt['PEOR_DESEMPEÑO_INTERNET'] = 1 - Data['DESEMPEÑO_INTERNET']
# Crear columnas binarias para cada valor en 'FAMI_TIENEINTERNET' con prefijo 'INTERNET'
zt = pd.get_dummies(zt, columns=['FAMI_TIENEINTERNET'], prefix='INTERNET')
# Filtrar las columnas que empiezan con 'INTERNET_'
internet_columns_test = [col for col in zt.columns if col.startswith('INTERNET_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[internet_columns_test] = zt[internet_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in internet_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
print("columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE")
# Reemplazar valores en las columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE para zt
zt['FAMI_EDUCACIONPADRE'] = zt['FAMI_EDUCACIONPADRE'].replace(reemplazos_educacion)
zt['FAMI_EDUCACIONMADRE'] = zt['FAMI_EDUCACIONMADRE'].replace(reemplazos_educacion)

# Crear columnas binarias para 'FAMI_EDUCACIONPADRE' en zt con prefijo 'EDUPADRE_'
zt = pd.get_dummies(zt, columns=['FAMI_EDUCACIONPADRE'], prefix='EDUPADRE')
# Crear columnas binarias para 'FAMI_EDUCACIONMADRE' en zt con prefijo 'EDUMADRE_'
zt = pd.get_dummies(zt, columns=['FAMI_EDUCACIONMADRE'], prefix='EDUMADRE')
# Filtrar las columnas que empiezan con 'EDUPADRE_' y 'EDUMADRE_'
zt_edupadre_columns = [col for col in zt.columns if col.startswith('EDUPADRE_')]
zt_edumadre_columns = [col for col in zt.columns if col.startswith('EDUMADRE_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[zt_edupadre_columns] = zt[zt_edupadre_columns].astype(int)
zt[zt_edumadre_columns] = zt[zt_edumadre_columns].astype(int)
# Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONPADRE' en zt
print("Columnas binarias para FAMI_EDUCACIONPADRE en zt:")
for col in zt_edupadre_columns:
    print(f"{col}:\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
# Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONMADRE' en zt
print("Columnas binarias para FAMI_EDUCACIONMADRE en zt:")
for col in zt_edumadre_columns:
    print(f"{col}:\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PAGOMATRICULAPROPIO
print("COLUMNA ESTU_PAGOMATRICULAPROPIO")
zt['ESTU_PAGOMATRICULAPROPIO'] = zt['ESTU_PAGOMATRICULAPROPIO'].fillna('ValorNulo')
zt['DESEMPEÑO_PAGOM'] = zt['ESTU_PAGOMATRICULAPROPIO'].map(desempeno_pagom_map)
zt['PEOR_DESEMPEÑO_PAGOM'] = 1 - Data['DESEMPEÑO_PAGOM']
# Crear columnas binarias para cada valor en 'ESTU_PAGOMATRICULAPROPIO' con prefijo 'MLAPROPIO'
zt = pd.get_dummies(zt, columns=['ESTU_PAGOMATRICULAPROPIO'], prefix='MLAPROPIO')
# Filtrar las columnas que empiezan con 'MLAPROPIO_'
mlapropio_columns_test = [col for col in zt.columns if col.startswith('MLAPROPIO_')]
# Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[mlapropio_columns_test] = zt[mlapropio_columns_test].astype(int)
# Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in mlapropio_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")



COLUMNA PERIODO
PERIODO_2018 (test.csv):
 0    1
1    0
2    0
3    0
4    0
5    1
6    0
7    1
8    0
9    1
Name: PERIODO_2018, dtype: int64

----------------------------------------

PERIODO_2019 (test.csv):
 0    0
1    0
2    0
3    1
4    0
5    0
6    1
7    0
8    1
9    0
Name: PERIODO_2019, dtype: int64

----------------------------------------

PERIODO_2020 (test.csv):
 0    0
1    1
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: PERIODO_2020, dtype: int64

----------------------------------------

PERIODO_2021 (test.csv):
 0    0
1    0
2    1
3    0
4    1
5    0
6    0
7    0
8    0
9    0
Name: PERIODO_2021, dtype: int64

----------------------------------------

--------------------------------------------------------
COLUMNA ESTU_PRGM_ACADEMICO
0          CIENCIAS SOCIALES Y HUMANIDADES
1    CIENCIAS ECONOMICAS Y ADMINISTRATIVAS
2          CIENCIAS NATURALES E INGENIERIA
3    CIENCIAS ECONOMICAS Y ADMINISTRATIVAS
4    CIENCIAS ECONOMICAS Y ADMINISTRATI

In [ ]:
#Aplicación de Modelo
# Verificar que las columnas en zt coincidan con las utilizadas en el entrenamiento
missing_columns = set(X.columns) - set(zt.columns)
if missing_columns:
    print(f"Las siguientes columnas faltan en zt: {missing_columns}")
    for col in missing_columns:
        zt[col] = 0

#Asegurar que las columnas estén en el mismo orden que en el entrenamiento
zt = zt[X.columns]

#Generar predicciones para zt
zt['RENDIMIENTO_GLOBAL_PRED'] = model.predict(zt)

# Crear el archivo de envío
# Suponiendo que el archivo test.csv tiene los IDs originales
test_ids = pd.read_csv("test.csv")[['ID']]

# Crear un diccionario de mapeo inverso para las predicciones
reemplazos_rendimiento_inverso = {
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
}

#Mapear las predicciones numéricas a valores textuales
zt['RENDIMIENTO_GLOBAL_TEXT'] = zt['RENDIMIENTO_GLOBAL_PRED'].map(reemplazos_rendimiento_inverso)

#Fusionar con los IDs originales
submission = pd.concat([test_ids, zt[['RENDIMIENTO_GLOBAL_TEXT']]], axis=1)

#Renombrar las columnas para el archivo de envío
submission.rename(columns={'RENDIMIENTO_GLOBAL_TEXT': 'RENDIMIENTO_GLOBAL'}, inplace=True)

#Verificar las primeras filas del archivo de envío
print("Primeras filas del archivo de envío:")
print(submission.head())

#Guardar el archivo de envío en formato CSV
submission.to_csv("my_submission.csv", index=False)

# Confirmar que el archivo fue creado correctamente
print("Archivo 'my_submission.csv' creado exitosamente.")

Primeras filas del archivo de envío:
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
Archivo 'my_submission.csv' creado exitosamente.


In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f my_submission.csv -m "Predicción con modelo CatBoost"

100% 4.05M/4.05M [00:00<00:00, 5.34MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia